In [1]:
!python -V

Python 3.10.13


In [1]:
import pandas as pd
import pickle
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
import pprint
from sklearn.metrics import mean_squared_error

The data for this work is locaated at [click to download](https://www.nyc.gov/site/tlc/about/tlc-trip-record-data.page)

The metadata can be found at [click here](https://www.nyc.gov/assets/tlc/downloads/pdf/data_dictionary_trip_records_yellow.pdf)

In [29]:
df = pd.read_parquet("yellow_tripdata_2024-01.parquet")
# df_2 = pd.read_parquet("yellow_tripdata_2024-02.parquet")

df.head().T

,0,1,2,3,4
VendorID,2,1,1,1,1
tpep_pickup_datetime,2024-01-01 00:57:55,2024-01-01 00:03:00,2024-01-01 00:17:06,2024-01-01 00:36:38,2024-01-01 00:46:51
tpep_dropoff_datetime,2024-01-01 01:17:43,2024-01-01 00:09:36,2024-01-01 00:35:01,2024-01-01 00:44:56,2024-01-01 00:52:57
passenger_count,1.0,1.0,1.0,1.0,1.0
trip_distance,1.72,1.8,4.7,1.4,0.8
RatecodeID,1.0,1.0,1.0,1.0,1.0
store_and_fwd_flag,N,N,N,N,N
PULocationID,186,140,236,79,211
DOLocationID,79,236,79,211,148
payment_type,2,1,1,1,1


In [30]:
# Findings about the data


def summarize_data(data_input, data_target=None, stat=True):
    """provides a complete overview of the data including the number of columns, data type, missing values and descriptive statistics and the sub-category per category"""

    if data_target is not None:
        data = pd.concat([data_input, data_target], axis=1)
    else:
        data = data_input

    print("-" * 50)
    print("""Preview of the data""")
    print("-" * 50)

    print("\n\n")
    # print(data.head().T, end="\n\n")
    pprint.pp(data.head(2).T, depth=6, width=120)

    print("-" * 50)
    print("""Summary of data columns: counts and dtype""")
    print("-" * 50)

    print("\n\n")
    print(data.info())

    print("\n\n")
    print("-" * 50)
    print("""Summary of number of NULLs per columns""")
    print("-" * 50)
    print(data.isna().sum())

    print("\n\n")
    print("-" * 50)
    print("""Summary of Number of Duplicates in the Data""")
    print("-" * 50)
    print(data.duplicated().sum())

    if stat:
        print("\n\n")
        print("-" * 50)
        print("""Summary of Statistics on Numerical Variables""")
        print("-" * 50)
        print(data.select_dtypes(include=["int64", "float64"]).describe().T)

        print("\n\n")
        print("-" * 50)
        print("""Summary of Statistics on Categorical Variables""")
        print("-" * 50)

        for feature in data.select_dtypes(include=["object"]).columns.to_list()[:]:
            print("\n\n")
            print("-" * 50)
            print(f"Summary of Statistics on Categorical Variables: {feature}")
            print("-" * 50)
            print(data[feature].value_counts(normalize=True))
            print(f"## number of unique subfeatures: {data[feature].nunique()} ")


summarize_data(df, stat=True)

--------------------------------------------------
Preview of the data
--------------------------------------------------



                                         0                    1
VendorID                                 2                    1
tpep_pickup_datetime   2024-01-01 00:57:55  2024-01-01 00:03:00
tpep_dropoff_datetime  2024-01-01 01:17:43  2024-01-01 00:09:36
passenger_count                        1.0                  1.0
trip_distance                         1.72                  1.8
RatecodeID                             1.0                  1.0
store_and_fwd_flag                       N                    N
PULocationID                           186                  140
DOLocationID                            79                  236
payment_type                             2                    1
fare_amount                           17.7                 10.0
extra                                  1.0                  3.5
mta_tax                                0.5 

In [21]:
df.dtypes

VendorID                          int32
tpep_pickup_datetime     datetime64[us]
tpep_dropoff_datetime    datetime64[us]
passenger_count                 float64
trip_distance                   float64
RatecodeID                      float64
store_and_fwd_flag               object
PULocationID                      int32
DOLocationID                      int32
payment_type                      int64
fare_amount                     float64
extra                           float64
mta_tax                         float64
tip_amount                      float64
tolls_amount                    float64
improvement_surcharge           float64
total_amount                    float64
congestion_surcharge            float64
Airport_fee                     float64
dtype: object

In [31]:
# Q1 = 19
len(df.dtypes.index.to_list())

19

## Data Preparation and Feature engineering

In [32]:
df["duration"] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime

# duration in minutes
df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

df.dtypes

VendorID                          int32
tpep_pickup_datetime     datetime64[us]
tpep_dropoff_datetime    datetime64[us]
passenger_count                 float64
trip_distance                   float64
RatecodeID                      float64
store_and_fwd_flag               object
PULocationID                      int32
DOLocationID                      int32
payment_type                      int64
fare_amount                     float64
extra                           float64
mta_tax                         float64
tip_amount                      float64
tolls_amount                    float64
improvement_surcharge           float64
total_amount                    float64
congestion_surcharge            float64
Airport_fee                     float64
duration                        float64
dtype: object

In [33]:
df.duration.describe()
# Q2 = 34.85

count    2.964624e+06
mean     1.561295e+01
std      3.485105e+01
min     -1.356667e+01
25%      7.150000e+00
50%      1.163333e+01
75%      1.868333e+01
max      9.455400e+03
Name: duration, dtype: float64

In [34]:
# Q3=97.7%
((df.duration >= 1) & (df.duration <= 60)).sum() / len(df.duration)

0.9778326020432945

In [35]:
df = df[(df.duration >= 1) & (df.duration <= 60)]

df = df.loc[:, ["duration", "PULocationID", "DOLocationID", "trip_distance"]]
df = pd.get_dummies(df, columns=["PULocationID", "DOLocationID"], drop_first=True)
df.head().T

,0,1,2,3,4
duration,19.8,6.6,17.916667,8.3,6.1
trip_distance,1.72,1.8,4.7,1.4,0.8
PULocationID_2,False,False,False,False,False
PULocationID_3,False,False,False,False,False
PULocationID_4,False,False,False,False,False
...,...,...,...,...,...
DOLocationID_261,False,False,False,False,False
DOLocationID_262,False,False,False,False,False
DOLocationID_263,False,False,False,False,False
DOLocationID_264,False,False,False,False,False


In [36]:
#Q4 = 515
len(df.columns.to_list())

518

In [2]:
def data_preprocessing(df):

    # calculate duration
    df["duration"] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)
    # filter duration
    df = df[(df.duration >= 1) & (df.duration <= 60)]

    # drop columns
    df = df.loc[:, ["duration", "PULocationID",
                    "DOLocationID", "trip_distance"]]

    # create dummies
    df = pd.get_dummies(
        df, columns=["PULocationID", "DOLocationID"], drop_first=True)

    return df


def split_data(data, target):
    X = data.drop(target, axis=1)
    y = data[target]
    return X, y


def train_model(X, y, model):
    model.fit(X, y)
    return model


def evaluate_model(model, X, y):
    y_pred = model.predict(X)
    return mean_squared_error(y, y_pred, squared=False)


df = pd.read_parquet("yellow_tripdata_2024-01.parquet")

df = data_preprocessing(df)

X, y = split_data(df, "duration")

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2898906 entries, 0 to 2964623
Columns: 518 entries, duration to DOLocationID_265
dtypes: bool(516), float64(2)
memory usage: 1.5 GB


In [6]:
df.isna().any().sum()

0

In [7]:
model = LinearRegression()

model = train_model(X[:100_000], y[:100_000], model)

In [8]:
# Q5 = 7.2
rmse = evaluate_model(model, X[:100_000], y[:100_000])
print(rmse)

7.205552728271093


/home/godwin/Desktop/Organised/python/python/MLOPs/.venv/lib/python3.10/site-packages/sklearn/metrics/_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


In [4]:
df = pd.read_parquet("yellow_tripdata_2024-02.parquet")

df = data_preprocessing(df)

X_, y_ = split_data(df, "duration")

In [5]:
#Q6=11.81
rmse = evaluate_model(model, X_[:10_000], y_[:10_000])
print(rmse)

ValueError: The feature names should match those that were passed during fit.
Feature names unseen at fit time:
- PULocationID_111
- PULocationID_187
- PULocationID_99
Feature names seen at fit time, yet now missing:
- DOLocationID_105
- PULocationID_105
- PULocationID_109
- PULocationID_156
- PULocationID_176
- ...
